In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/llm-classification-finetuning/train.csv')
train_df.head()

In [ ]:
import numpy as np

# Create a label column: 0 = A wins, 1 = B wins, 2 = tie
def get_label(row):
    if row['winner_model_a'] == 1:
        return 0
    elif row['winner_model_b'] == 1:
        return 1
    elif row['winner_tie'] == 1:
        return 2
    else:
        return -1  # catch errors if any

train_df['label'] = train_df.apply(get_label, axis=1)

In [ ]:
train_df.head()

In [ ]:
train_df['prompt_len'] = train_df['prompt'].apply(lambda x: len(str(x).split()))
train_df['a_len'] = train_df['response_a'].apply(lambda x: len(str(x).split()))
train_df['b_len'] = train_df['response_b'].apply(lambda x: len(str(x).split()))
train_df['len_diff'] = train_df['a_len'] - train_df['b_len']


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

X = train_df[['prompt_len', 'a_len', 'b_len', 'len_diff']]
y = train_df['label']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

val_probs = model.predict_proba(X_val)
print("Log Loss:", log_loss(y_val, val_probs))


In [ ]:
test_df = pd.read_csv('/kaggle/input/llm-classification-finetuning/test.csv')

# Create same features
test_df['prompt_len'] = test_df['prompt'].apply(lambda x: len(str(x).split()))
test_df['a_len'] = test_df['response_a'].apply(lambda x: len(str(x).split()))
test_df['b_len'] = test_df['response_b'].apply(lambda x: len(str(x).split()))
test_df['len_diff'] = test_df['a_len'] - test_df['b_len']

# Predict
test_X = test_df[['prompt_len', 'a_len', 'b_len', 'len_diff']]
test_probs = model.predict_proba(test_X)

# Prepare submission
submission = pd.DataFrame({
    'id': test_df['id'],
    'winner_model_a': test_probs[:, 0],
    'winner_model_b': test_probs[:, 1],
    'winner_tie': test_probs[:, 2],
})

# ✅ Save to correct location
submission.to_csv('submission.csv', index=False)
